In [20]:
import multiprocessing
import mne
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import scipy.io as sio
import sys
from scipy.signal import hilbert
import h5py
import neurokit2 as nk
import pandas as pd
import antropy as ant
sys.path.insert(0, "C:/Users/Antoine/github/MEG_pareidolia/python_scripts/Functions")
from MEG_pareidolia_utils import *
import PARAMS
from PARAMS import *

In [2]:
main_df = pd.read_csv('df_ALL_metadata_MEG_sub00to11_epo_long_Higuchi_DFA.csv')

In [3]:
main_df

,trials,electrodes,index,DiffEn,FI,Hjorth,KFD,PEn,PFD,RR,...,exp,Task,DFA_delta,DFA_theta,DFA_alpha,DFA_low_beta,DFA_high_beta,DFA_gamma1,DFA_gamma2,Higuchi
0,0,0,0,-28.467058,0.597310,2.328684,3.286454,0.999980,1.004388,0.117301,...,0.946508,pareidolia,1.828560,1.720403,1.707759,1.544672,1.526756,1.346672,1.329637,1.572093
1,0,1,0,-28.373809,0.590333,2.216524,3.487840,0.999995,1.004792,0.134195,...,0.888771,pareidolia,1.803569,1.755877,1.693964,1.566307,1.533651,1.333845,1.324667,1.605892
2,0,2,0,-28.199392,0.599369,2.315645,3.324723,0.999876,1.004458,0.121311,...,0.865055,pareidolia,1.829340,1.760806,1.691584,1.571995,1.563555,1.371460,1.326044,1.595664
3,0,3,0,-27.975919,0.571585,2.236874,3.499786,0.999277,1.004704,0.132936,...,0.891698,pareidolia,1.885524,1.759546,1.671845,1.592450,1.566255,1.373067,1.292855,1.615484
4,0,4,0,-27.888237,0.540041,2.117262,3.228364,0.999705,1.004845,0.151575,...,0.858703,pareidolia,1.900989,1.757578,1.665750,1.625097,1.585245,1.381392,1.306705,1.633110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963085,51,265,0,-28.363491,0.630667,2.692908,3.065412,0.999996,1.004207,0.092303,...,1.145132,pareidolia,1.877691,1.728242,1.706400,1.524623,1.477893,1.347118,1.296460,1.566459
963086,51,266,0,-28.140529,0.657629,2.938791,3.246132,0.999998,1.004004,0.076541,...,0.972873,pareidolia,1.853897,1.714536,1.707499,1.560181,1.473148,1.346363,1.315242,1.565397
963087,51,267,0,-28.451383,0.517814,1.961774,3.562668,0.999994,1.005785,0.176946,...,0.774693,pareidolia,1.873505,1.723548,1.735867,1.565924,1.491144,1.355477,1.306770,1.648118
963088,51,268,0,-28.772145,0.412404,1.570937,4.035596,0.999980,1.007144,0.281648,...,0.612180,pareidolia,1.857247,1.712014,1.735834,1.548540,1.460907,1.321056,1.294421,1.679246


In [5]:
import pandas as pd

task = 'pareidolia'
# Initialize a list to store all DFA DataFrames
all_higuchi_dfs = []
list_subj = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']
run_list = [1, 2, 3, 4, 5, 6]
# Loop through each subject and run to load DFA DataFrames
for subj in list_subj:
    for run in run_list:
        try:
            # Construct the path to the saved DFA DataFrame
            _, dfa_path = get_pareidolia_bids(FOLDERPATH, subj, task, run, stage="array_comp_dfa")

            # Load the DFA DataFrame
            dfa_df = np.load(dfa_path+'.npy')

            # Create DF from the array (trials, channels) with one column for electrode, one for trial and one for higuchi
            higuchi_df = pd.DataFrame(dfa_df, columns=[i for i in range(dfa_df.shape[1])])
            higuchi_df['trials'] = higuchi_df.index
            
            higuchi_df = pd.melt(higuchi_df, id_vars=['trials'], var_name='electrodes', value_name='dfa')
            higuchi_df['participant'] = subj
            higuchi_df['bloc'] = run
            all_higuchi_dfs.append(higuchi_df)
        except FileNotFoundError:
            print(f'File not found for subject {subj} and run {run}')

# Assuming all_dfa_dfs is a list of your loaded and column-renamed DFA DataFrames
concatenated_df = pd.concat(all_higuchi_dfs)

# Drop duplicates from both the main and DFA DataFrames
main_df.drop_duplicates(subset=['participant', 'bloc', 'trials', 'electrodes'], inplace=True)
concatenated_df.drop_duplicates(subset=['participant', 'bloc', 'trials', 'electrodes'], inplace=True)

# Ensure that the indexes are reset
main_df.reset_index(drop=True, inplace=True)
concatenated_df.reset_index(drop=True, inplace=True)

# Convert columns to integers in both DataFrames

main_df['participant'] = main_df['participant'].astype(int)
main_df['bloc'] = main_df['bloc'].astype(int)
main_df['trials'] = main_df['trials'].astype(int)
main_df['electrodes'] = main_df['electrodes'].astype(int)

concatenated_df['participant'] = concatenated_df['participant'].astype(int)
concatenated_df['bloc'] = concatenated_df['bloc'].astype(int)
concatenated_df['trials'] = concatenated_df['trials'].astype(int)
concatenated_df['electrodes'] = concatenated_df['electrodes'].astype(int)

# Now merge the DataFrames
merged_df = pd.merge(main_df, concatenated_df, on=['participant', 'bloc', 'trials', 'electrodes'], how='inner')



File not found for subject 00 and run 6
File not found for subject 11 and run 4


In [6]:
merged_df.to_csv('df_ALL_metadata_MEG_sub00to11_epo_long_last.csv', index=False)

In [19]:
data['freq_range']

0         [2.0, 100.0]
1         [2.0, 100.0]
2         [2.0, 100.0]
3         [2.0, 100.0]
4         [2.0, 100.0]
              ...     
987115    [2.0, 100.0]
987116    [2.0, 100.0]
987117    [2.0, 100.0]
987118    [2.0, 100.0]
987119    [2.0, 100.0]
Name: freq_range, Length: 987120, dtype: object

In [13]:
data = pd.read_csv(
    "df_ALL_metadata_MEG_sub00to11_epo_RT_ALL_new.csv"
)
print(data.columns)
data_participant = (
    data.groupby(by="participant").mean().reset_index()
)
data_participant = data_participant[
    ["n_obj", "parei", "participant"]
]
print(data_participant.columns)

Index(['trials', 'electrodes', 'index', 'DiffEn', 'FI', 'Hjorth', 'KFD', 'PEn',
       'PFD', 'RR', 'SFD', 'SVDEn', 'ShanEn', 'SpEn', 'bloc', 'participant',
       'FD', 'FD_class', 'n_obj', 'n_obj_class', 'parei', 'contrast',
       'earlyVSlate', 'delta', 'theta', 'alpha', 'low_beta', 'high_beta',
       'gamma1', 'gamma2', 'LZ', 'Unnamed: 0', 'offset', 'exp', 'freq_range',
       'freq_res', 'DAT'],
      dtype='object')


TypeError: Could not convert [2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0][2.0, 100.0

In [11]:
df_

,electrodes,delta,theta,alpha,low_beta,high_beta,gamma1,gamma2,trials
0,0,-243.571751,-245.705630,-246.857479,-248.841180,-247.692193,-254.375885,-256.165953,58
1,1,-242.134824,-244.721674,-245.366567,-249.367676,-246.635281,-252.492020,-254.397587,58
2,2,-240.349446,-243.502530,-244.297899,-248.776083,-246.499051,-252.138320,-255.154492,58
3,3,-238.941420,-242.365902,-243.346750,-248.608424,-246.406700,-252.319643,-255.024079,58
4,4,-237.854990,-241.875899,-242.660892,-247.201135,-245.699475,-253.119401,-255.162566,58
...,...,...,...,...,...,...,...,...,...
265,265,-242.506131,-244.315383,-248.286910,-249.384519,-248.443149,-254.924121,-255.775704,58
266,266,-238.607868,-238.538877,-238.723098,-241.626496,-244.932862,-250.364542,-253.761203,58
267,267,-241.868575,-244.284189,-243.850658,-248.609726,-249.193133,-253.395839,-255.585487,58
268,268,-249.669368,-250.539250,-252.370971,-253.763635,-252.756192,-255.387940,-256.141848,58
